# The basics
In this notebook we show how to calculate:
- the SNR
- the Fisher Matrix
- the covariance matrix

and finally we estimate the uncertainties on the parameter from parameter estimation. <br >
(as a general rule, auxiliary functions start with an underscore, all other functions start with a capital letter)

Start by loading the code

In [1]:
using GW

### First of all, we need to decide a waveform model and a detector configuration: <br >

In [2]:
# check the waveforms available with this function:
_available_waveforms()

7-element Vector{String}:
 "TaylorF2"
 "PhenomD"
 "PhenomHM"
 "PhenomD_NRTidal"
 "PhenomNSBH"
 "PhenomXAS"
 "PhenomXHM"

A short description of these possibilites:
- TaylorF2 is a very basic waveform, not useful
- PhenomD is a good compromise between accuracy and speed for this tutorial. Valid only for BBH
- PhenomXHM is the most accurate waveform. The recommended one for the final results, valid only for BBH (takes a while to compile, order 5 minutes)
- PhenomHM simpler than PhenomXHM, but still accurate. Valid only for BBH
- PhenomD_NRTidal is the PhenomD waveform with the addition of the tidal effects. Valid only for BNS
- PhenomNSBH is the waveform valid for NSBH
- PhenomXAS a more refined and faster version of PhenomD. Valid only for BBH

Given these possibilites, the waveform can be set in two ways:

In [3]:
# First possibility: use the waveform directly
wfPhenomD = PhenomD();

# Second possibility: use the auxiliary function _available_waveforms() 
wfPhenomD = _available_waveforms("PhenomD");

Similarly, we can check the available detectors:

In [4]:
# check the detectors available with this function
_available_detectors()

The detectors available are: 


11-element Vector{String}:
 "CE1Id, 40km"
 "CE2NM, 20km"
 "CE2NSW, 20km"
 "ETS, 10km"
 "ETLS, 10km"
 "ETMR, 10km"
 "ETLMR, 10km"
 "LIGO_L"
 "LIGO_H"
 "VIRGO"
 "KAGRA"

For some CE/ET configurations we report the armlength in km <br >
NB: This is not needed to call the detector e.g. _available_detectors("CE1Id") <br > <br >

Again, the detector can be set in two different ways:

In [5]:
# First possibility: use the detector directly
CE_1= CE1Id

# Second possibility: use the function _available_detectors()
CE_1 = _available_detectors("CE1Id");

We can also define networks:

In [6]:
# define two other detectors
CE_2 = CE2NM
ET = ETS

# put them all together in an array to define a network
network = [CE_1, CE_2, ET];

### Then we define an event by specifying its parameters:

In [7]:
mc = 30.0 # chirp mass at the detector frame in solar masses
η = 0.2 # symmetric mass ratio, dimensionless 
χ_1 = 0.1 # dimensionless spin of the most massive BH
χ_2 = 0.2 # dimensionless spin of the least massive BH
dL = 1.0 # luminosity distance in Gpc
θ = 45. * pi/180 # latitude in radians
ϕ = 45. * pi/180 # longitude in radians
ι = 45. * pi/180 # inclination angle in radians
ψ = 45. * pi/180 # polarisation angle in radians
tcoal = 0.0 # time of coalescence in fraction of a day, [0,1]
Φ_coal = 0.0; # phase of coalescence in radians

We should then define an array containing the parameters:

In [8]:
parameters = [mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal];

### Here we compute the SNR for our network

The arguments of the SNR function are:
- the waveform
- a network of detectors (can either be a list of detectors or a single detector)
- the parameters: mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal 

Notice that the SNR DOES NOT depend on the phase to coalescence Φ_coal <br > 
so the SNR function does not need it (if you include it, you will get an error)

In [9]:
snr = SNR(wfPhenomD, network, parameters[1:10]...)

777.6100165434572

### The next step is to compute the Fisher matrix

As above, the first call might be slow (up to a minute or so) because the code needs to be compiled first), each subsequent call will be blitzing fast <br >

The FisherMatrix function takes the following parameters:
- the waveform
- a network of detectors
- the parameters: mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal

As above the order of the paramters is important! <br >

In [10]:
# We can now move to the Fisher matrix of that event but choose a network of detectors
myfisher = FisherMatrix(wfPhenomD, network, parameters...);

With this setup (BBH), the fisher is a 11x11 matrix, the order is the same as the inputs. 
The units of the fisher matrix are the inverse of the square of the units of the parameters. In particular:
- the chirp mass is in solar masses
- the symmetric mass ratio is dimensionless
- the spins are dimensionless
- the luminosity distance is in Gpc 
- the angles are in radians
- the time of coalescence is in days
- the phase to coalescence is in radians

All uncertainties estimated below will be in the same units. The only exception to this rule is the time of coalescence, which is converted to seconds.

### Then we calculate the covariance matrix (by inverting the Fisher matrix)

In [11]:
mycovariance = CovMatrix(myfisher);

Inversion successful
Inversion error: 3.5037483342146415e-8



Finally we forecast the parameter uncertainties

In [12]:
myerrors = Errors(mycovariance)
parameters_string = ["mc", "η", "χ_1", "χ_2", "dL", "θ", "ϕ", "ι", "ψ", "tcoal", "Φ_coal"]

for i in eachindex(myerrors)
    println("The error on $(parameters_string[i]) is $(myerrors[i])")
end

The error on mc is 0.0008503378690617058
The error on η is 0.0008092744312426696
The error on χ_1 is 0.01593054906342642
The error on χ_2 is 0.044362965472316176
The error on dL is 0.007751590982753607
The error on θ is 0.00043375280052513926
The error on ϕ is 0.001916780891173378
The error on ι is 0.009008652208758699
The error on ψ is 0.011686116325357748
The error on tcoal is 0.0006516513966047323
The error on Φ_coal is 0.05295043311349371


For the angles, we convert to get a sky area for the event using the SkyArea function with inputs: 
- the covariance matrix
- the latitude 
- some credible interval (in %, default is 90%)

In [13]:
sky_area = SkyArea(mycovariance, θ, percent_level = 90)

0.024299845476065625

The angle up here is in grad, and represents the 90% credible interval 